# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,klyuchi,52.2667,79.1667,-9.81,89,7,2.54,RU,1732724974
1,1,rouyn-noranda,48.2398,-79.0288,-2.95,86,100,3.60,CA,1732724976
2,2,jixi,45.3000,130.9667,-15.39,97,38,1.29,CN,1732724977
3,3,albany,42.6001,-73.9662,6.86,59,70,3.13,US,1732724418
4,4,abbadia san salvatore,42.8821,11.6695,11.29,88,100,1.01,IT,1732724980
...,...,...,...,...,...,...,...,...,...,...
563,563,opuwo,-18.0607,13.8400,29.05,16,0,7.73,NaN,1732725665
564,564,awjilah,29.1081,21.2869,15.49,45,6,3.61,LY,1732725666
565,565,vavoua,7.3819,-6.4778,31.58,29,65,1.91,CI,1732725667
566,566,barra de santo antonio,-9.4047,-35.5072,27.26,62,60,5.41,BR,1732725668


In [11]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     568 non-null    int64  
 1   City        568 non-null    object 
 2   Lat         568 non-null    float64
 3   Lng         568 non-null    float64
 4   Max Temp    568 non-null    float64
 5   Humidity    568 non-null    int64  
 6   Cloudiness  568 non-null    int64  
 7   Wind Speed  568 non-null    float64
 8   Country     565 non-null    object 
 9   Date        568 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 44.5+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
map_plot = city_data_df.hvplot.points(
    'Lng',  # Longitude column
    'Lat',  # Latitude column

    size='Humidity',  # Column from above data set
    geo=True,  # Copy from Geo Airports
    tiles='OSM',
    frame_width=800,  # Width
    frame_height=600,# Height
color = "City",
hover_cols=["City", "Country", "Max Temp"]
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [1]:

# Narrow down cities that fit criteria +drop any results with null values
min_temp = 70
max_temp = 80
wind_speed= 10

# Drop any rows with null values
city_data2_df = city_data_df.dropna()

mask = (city_data2_df["Max Temp"] >= min_temp) & (city_data2_df["Max Temp"] < max_temp) & (city_data2_df["Wind Speed"] < wind_speed)
city_data_df = city_data2_df.loc[mask].reset_index(drop=True)

# Display sample

city_data2_df.info()


NameError: name 'city_data_df' is not defined

### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
hotel_rows = []

for index, row in city_data2_df.iterrows():
    # Similar to Weather Data activity
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    country = row["Country"]

    # Set the  type of place(Hotel) get from website for geoplots
    categories = "accommodation.hotel"
    radius = 10000 # 10km

    # Set what we are looking for
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5

    # set up  dictionary
    hotel_dict = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key,
        "city":city
    }

    # Set base URL-again from website
    base_url = "https://api.geoapify.com/v2/places"

    # Run request
    try:
        response = requests.get(base_url, params=hotel_dict)
        # print(response.status_code)
        data = response.json()

        # Print the results
        results = data.get("features", [])

        # Error Handling
        if len(results) > 0:
            place = results[0]

            # normal data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")

            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
            "city": city,
            "country": country,
            "latitude": latitude,
            "longitude": longitude,
            "address": None,
            "name": None,
            "distance": None,
            "elevation": None,
            "website": None
        }

    # send data to hotel list
    hotel_rows.append(hotel_row)

    # pull into a data frame
hotel_df = pd.DataFrame(hotel_rows)
print(hotel_df.head())


                    city country  latitude  longitude  \
0                klyuchi      RU   52.2667    79.1667   
1          rouyn-noranda      CA   48.2398   -79.0288   
2                   jixi      CN   45.3000   130.9667   
3                 albany      US   42.6001   -73.9662   
4  abbadia san salvatore      IT   42.8821    11.6695   

                                             address  \
0                                               None   
1  Best Western Plus Hotel Albert, Avenue Princip...   
2  Hanting Hotel, 兴国路, Hongjunlu Subdistrict, 158...   
3                                               None   
4  Hotel Giardino, Via Primo Maggio, 63, 53021 Ab...   

                             name  distance  elevation website  
0                            None       NaN        NaN    None  
1  Best Western Plus Hotel Albert     617.0        NaN    None  
2                            汉庭酒店     504.0        NaN    None  
3                            None       NaN        NaN    No

In [25]:

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
#Empty hotel row
hotel_rows = []
#start of For Loop
for index, row in hotel_df.iterrows():
    # from the Weather CSV
    city = row["city"]
    country = row["country"]
    latitude = row["latitude"]
    longitude = row["longitude"]

    # tell what we are looking for
    categories = "accommodation.hotel"
    radius = 10000 # 10km

    # what am I looking for in greater detail
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 5

    # Turn what im looking for into an organize dictonary
    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias":bias,
        "apiKey":geoapify_key
    }

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Run request us a try/if/then ( similar to a if else ifelse statment)
    try:
        response = requests.get(base_url, params=params)
        # print(response.status_code)
        data = response.json()

        # Print the results
        results = data.get("features", [])

        # Resiliency/Error Handling
        if len(results) > 0:
            place = results[0]

            # normalize data
            address = place.get("properties", {}).get("formatted")
            name = place.get("properties", {}).get("name")
            distance = place.get("properties", {}).get("distance")
            elev = place.get("properties", {}).get("ele")
            website = place.get("properties", {}).get("website")

            # return object
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": address,
                "name": name,
                "distance": distance,
                "elevation": elev,
                "website": website
            }
        else:
            hotel_row = {
                "city": city,
                "country": country,
                "latitude": latitude,
                "longitude": longitude,
                "address": None,
                "name": None,
                "distance": None,
                "elevation": None,
                "website": None
            }
    except Exception as e:
        print(e)
        hotel_row = {
            "city": city,
            "country": country,
            "latitude": latitude,
            "longitude": longitude,
            "address": None,
            "name": None,
            "distance": None,
            "elevation": None,
            "website": None
        }

    # make hotel list
    hotel_rows.append(hotel_row)
    hotel_df = pd.DataFrame(hotel_rows)
hotel_df

,city,country,latitude,longitude,address,name,distance,elevation,website
0,klyuchi,RU,52.2667,79.1667,None,None,NaN,NaN,None
1,rouyn-noranda,CA,48.2398,-79.0288,"Best Western Plus Hotel Albert, Avenue Princip...",Best Western Plus Hotel Albert,617.0,NaN,None
2,jixi,CN,45.3000,130.9667,"Hanting Hotel, 兴国路, Hongjunlu Subdistrict, 158...",汉庭酒店,504.0,NaN,None
3,albany,US,42.6001,-73.9662,None,None,NaN,NaN,None
4,abbadia san salvatore,IT,42.8821,11.6695,"Hotel Giardino, Via Primo Maggio, 63, 53021 Ab...",Hotel Giardino,86.0,NaN,None
...,...,...,...,...,...,...,...,...,...
560,kyshtovka,RU,56.5550,76.6271,None,None,NaN,NaN,None
561,awjilah,LY,29.1081,21.2869,"Awjilah Ajdabyia Road, Ajdabiya, Libya",None,6665.0,NaN,None
562,vavoua,CI,7.3819,-6.4778,"Hôtel Sougoun, A5, Goulaonfla, Côte d'Ivoire",Hôtel Sougoun,1287.0,NaN,None
563,barra de santo antonio,BR,-9.4047,-35.5072,"Resort Fazenda Fiore, Rodovia AL 101 Norte, Pa...",Resort Fazenda Fiore,5684.0,NaN,None


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",

    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "city",
    hover_cols=["city", "country", "name", "address"]  # Add 'city' to the tooltip
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (city,country,name,address)